In this Notebook I intend to perform basic data exploration of the databases provided by the MinTIC for the DS4A Final project.

I already made copies of each table in CSV format and manipulated them a little as to have unique columns plus an additional column of year. This way, I think, I may be able to further manipulate and explore them in Pandas

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
mdm = pd.read_csv("../data/MDM_2016_2019.csv")
comp_dim = pd.read_csv("../data/Componentes_Dimensiones_2016_2019.csv", header=1)
investments = pd.read_csv("../data/BD_MinTIC_2010_2020.csv")
pd.options.display.max_columns = 100

First we are interested in looking for the basic structure of each table, beginning by the MDM table:

In [36]:
mdm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Anio_corte                   4404 non-null   int64  
 1   Municipio                    4404 non-null   object 
 2   Código                       4404 non-null   int64  
 3   Departamento                 4404 non-null   object 
 4   Grupo_dotaciones             4404 non-null   object 
 5   Categoría_de_ruralidad       4404 non-null   object 
 6   MDM                          4404 non-null   float64
 7   Puesto_MDM_Grupo_Dotaciones  4404 non-null   int64  
 8   Puesto_MDM_Nacional          4404 non-null   int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 309.8+ KB


The dtype of each column is as expected.

So, next we will examine the numeric variables of this table, looking for its minimum and maximum values, as well as a basic grasp on its distributions. Though we won't treat the year nor the municipality codes as numeric values in further analyses, it is still interesting to see which are the minimum and maximum values these varibles take.

In [37]:
mdm.describe()

,Anio_corte,Código,MDM,Puesto_MDM_Grupo_Dotaciones,Puesto_MDM_Nacional
count,4404.000000,4404.000000,4404.000000,4404.000000,4404.000000
mean,2017.500000,37671.950954,50.372279,108.092643,551.000000
std,1.118161,25705.991227,10.170328,63.422778,317.867283
min,2016.000000,5001.000000,2.033311,1.000000,1.000000
25%,2016.750000,15646.000000,43.950000,53.000000,276.000000
50%,2017.500000,25781.000000,50.209631,108.000000,551.000000
75%,2018.250000,63302.000000,56.550113,163.000000,826.000000
max,2019.000000,99773.000000,86.816932,218.000000,1101.000000


Finally, we are interested in creating a synthetic primary key for this table, in order to merge its information to the other tables in further analyses:

In [38]:
mdm["id"] = mdm["Anio_corte"].astype("string")+mdm["Código"].astype("string")
mdm

,Anio_corte,Municipio,Código,Departamento,Grupo_dotaciones,Categoría_de_ruralidad,MDM,Puesto_MDM_Grupo_Dotaciones,Puesto_MDM_Nacional,id
0,2016,MEDELLÍN,5001,ANTIOQUIA,C,Ciudades_y_aglomeraciones,80.780000,1,3,20165001
1,2016,ABEJORRAL,5002,ANTIOQUIA,G3,Rural,49.480000,76,445,20165002
2,2016,ABRIAQUÍ,5004,ANTIOQUIA,G3,Rural_disperso,44.940000,132,678,20165004
3,2016,ALEJANDRÍA,5021,ANTIOQUIA,G2,Rural,54.940000,66,232,20165021
4,2016,AMAGÁ,5030,ANTIOQUIA,G2,Intermedios,51.880000,98,334,20165030
...,...,...,...,...,...,...,...,...,...,...
4399,2019,TARAIRA,97666,VAUPES,G2,Rural_disperso,42.470492,207,978,201997666
4400,2019,PUERTO_CARREÑO,99001,VICHADA,G1,Rural_disperso,57.566808,167,425,201999001
4401,2019,LA_PRIMAVERA,99524,VICHADA,G4,Rural_disperso,56.287012,60,492,201999524
4402,2019,SANTA_ROSALÍA,99624,VICHADA,G2,Rural_disperso,48.886473,188,825,201999624


Also, we would like to let every table in lower case, and to remove special characters like tildes and 'ñ's:


In [39]:
dict_ = {
    'á' : 'a',
    'é' : 'e',
    'í' : 'i',
    'ó' : 'o',
    'ú' : 'u',
    'ü' : 'u',
    'ñ' : 'ni'
}

def remove_tildes(ser):
    new_list = []
    for element in list(ser):
        new_item = []
        new_item.append([dict_.get(char, char) for char in element])
        new_item = ''.join(new_item[0])
        new_list.append(new_item)
    return new_list

In [40]:
mdm.columns = remove_tildes(mdm.columns.str.lower())
mdm.columns

Index(['anio_corte', 'municipio', 'codigo', 'departamento', 'grupo_dotaciones',
       'categoria_de_ruralidad', 'mdm', 'puesto_mdm_grupo_dotaciones',
       'puesto_mdm_nacional', 'id'],
      dtype='object')

In [41]:
mdm["municipio"] = remove_tildes(mdm["municipio"].str.lower())
mdm["departamento"] = remove_tildes(mdm["departamento"].str.lower())
mdm["grupo_dotaciones"] = mdm["grupo_dotaciones"].str.lower()
mdm["categoria_de_ruralidad"] = remove_tildes(mdm["categoria_de_ruralidad"].str.lower())


Also, we would like to remove tildes from the names of municipalities and departments:

Now we will repeat the same steps for the MinTIC investments table:

In [42]:
investments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63361 entries, 0 to 63360
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ANIO_CORTE             63361 non-null  int64 
 1   FECHA_CORTE            63361 non-null  object
 2   DEPARTAME_NOMBRE       63361 non-null  object
 3   DEPARTAME_COD          63361 non-null  object
 4   MUNICIPIO_NOMBRE       63361 non-null  object
 5   MUNICIPIO_COD          63361 non-null  int64 
 6   INICIATIVA             63361 non-null  object
 7   VICEMINISTERIO         63361 non-null  object
 8   INDICADOR              63361 non-null  object
 9   DESCRIPCION_INDICADOR  63361 non-null  object
 10  INVERSION              63361 non-null  int64 
 11  BENEFICIARIOS          63361 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 5.8+ MB


As before, the column dtypes are as expected, except for the `DEPARTAME_COD` column, which appears as an `object` dtype, while we expected an integer dtype. Also the `FECHA_CORTE` column should be a `data` dtype, but it is an `object`. Let's examine first the `DEPARTAME_COD` column:

In [43]:
investments.columns = remove_tildes(investments.columns.str.lower())
investments.columns

Index(['anio_corte', 'fecha_corte', 'departame_nombre', 'departame_cod',
       'municipio_nombre', 'municipio_cod', 'iniciativa', 'viceministerio',
       'indicador', 'descripcion_indicador', 'inversion', 'beneficiarios'],
      dtype='object')

In [44]:
investments["fecha_corte"] = pd.to_datetime(investments["fecha_corte"])

for col in investments.columns:
    if investments[col].dtype == 'object':
        investments[col] = remove_tildes(investments[col].str.lower())

In [45]:
investments["departame_cod"].unique()

array(['11', '5', '76', '25', '8', '52', '70', '13', '73', '15', '66',
       '99', '68', '23', '63', '41', '44', '94', '91', '17', '19', '54',
       '95', '20', '50', '18', '27', '86', '47', '81', '97', '85', '88',
       '0', 'in'], dtype=object)

Ok, so the reason it is treated as an `object` dtype is that 'IN' value.

In [46]:
investments[investments["departame_cod"]=="in"]

,anio_corte,fecha_corte,departame_nombre,departame_cod,municipio_nombre,municipio_cod,iniciativa,viceministerio,indicador,descripcion_indicador,inversion,beneficiarios
17655,2020,2020-01-20,internacional,in,internacional,1123,habilidades_digitales_ciencia_de_datos,transformacion,colombianos_formados_en_ciencia_de_datos,colombianos_formados_en_ciencia_de_datos,16118564,4
42812,2020,2020-12-31,internacional,in,internacional,1123,ciudadania_digital,transformacion,numero_de_formaciones_en_competencias_digitales,se_mide_el_avance_de_las_certificaciones_entre...,0,408
60182,2020,2020-01-20,internacional,in,internacional,1123,habilidades_digitales_ciencia_de_datos,transformacion,colombianos_formados_en_ciencia_de_datos,colombianos_formados_en_ciencia_de_datos,0,14


We see now that it is indeed a special code used for International investment projects. Perhaps it would be a good idea to try and think whether it would be a good idea to replace this code with a particular number (as to treat the entire column as an integer) or to let it as is and treat it as an `object` dtype. 

Next we will examine the numeric variables (again, as before):

In [47]:
investments.describe()

,anio_corte,municipio_cod,inversion,beneficiarios
count,63361.000000,63361.000000,6.336100e+04,6.336100e+04
mean,2017.484352,38704.559051,1.936421e+08,9.054179e+02
std,3.145821,26396.793814,3.279701e+09,3.508134e+04
min,1990.000000,0.000000,0.000000e+00,0.000000e+00
25%,2015.000000,15676.000000,0.000000e+00,0.000000e+00
50%,2019.000000,25839.000000,0.000000e+00,1.000000e+00
75%,2020.000000,66572.000000,2.568316e+07,2.500000e+01
max,2021.000000,99773.000000,7.735770e+11,7.592871e+06


In [48]:
print(f'Número de municipios registrados en MDM: {len(mdm["codigo"].unique())}')
print(f'Número de municipios registrados en BD_MinTIC: {len(investments["municipio_cod"].unique())}')

Número de municipios registrados en MDM: 1101
Número de municipios registrados en BD_MinTIC: 1146


According to the `describe` results of the MinTIC investments table, we can see that there are municipality codes that doesn't appear on the MDM table. We can see this because in the MDM table the minimum code was 5001, while in the investments table the minimum value is 0. This is further corroborated examining the number of codes appearing in each table, with a difference of 45 between them. In the next cell we look for the differing municipalities:

In [49]:
investments[~investments["municipio_cod"].isin(mdm["codigo"])].dropna()[["departame_nombre","municipio_nombre"]].drop_duplicates().sort_values("departame_nombre")

,departame_nombre,municipio_nombre
46,amazonas,tarapaca
82,amazonas,puerto_arica
115,amazonas,puerto_alegria
116,amazonas,el_encanto
7097,amazonas,la_victoria
748,amazonas,la_chorrera
7079,amazonas,puerto_santander
3859,amazonas,la_pedrera
4919,amazonas,miriti_-_parana
46298,antioquia,gobernacion


Now let's try the other way around and test whether there are municipalities represented in the MDM database but not present in the MinTIC investments database:

In [50]:
mdm[~mdm["codigo"].isin(investments["municipio_cod"])].dropna()[["departamento","municipio"]].drop_duplicates().sort_values("departamento")

,departamento,municipio


Finally, again we are interested in creating a synthetic primary key for the investment table, formed by the year and the municipality code:

In [51]:
investments["id"] = investments["anio_corte"].astype("string")+investments["municipio_cod"].astype("string")
investments

,anio_corte,fecha_corte,departame_nombre,departame_cod,municipio_nombre,municipio_cod,iniciativa,viceministerio,indicador,descripcion_indicador,inversion,beneficiarios,id
0,2020,2020-12-31,bogota,11,bogota_d.c.,11001,asesorias_en_teletrabajo,transformacion,numero_de_personas_impactadas_con_la_difusion_...,numero_de_personas_impactadas_con_la_difusion_...,773577049710,606,202011001
1,2020,2020-12-31,antioquia,5,medellin,5001,asesorias_en_teletrabajo,transformacion,numero_de_personas_impactadas_con_la_difusion_...,numero_de_personas_impactadas_con_la_difusion_...,114887680650,90,20205001
2,2020,2020-12-31,antioquia,5,apartado,5045,asesorias_en_teletrabajo,transformacion,numero_de_personas_impactadas_con_la_difusion_...,numero_de_personas_impactadas_con_la_difusion_...,70209138175,55,20205045
3,2020,2020-12-31,valle_del_cauca,76,cali,76001,asesorias_en_teletrabajo,transformacion,numero_de_personas_impactadas_con_la_difusion_...,numero_de_personas_impactadas_con_la_difusion_...,59996899895,47,202076001
4,2020,2020-12-31,cundinamarca,25,villapinzon,25873,asesorias_en_teletrabajo,transformacion,numero_de_personas_impactadas_con_la_difusion_...,numero_de_personas_impactadas_con_la_difusion_...,47231602045,37,202025873
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63356,2021,2021-02-12,putumayo,86,leguizamo,86573,vende_en_linea,transformacion,no_asociado,no_asociado,0,1,202186573
63357,2021,2021-02-12,putumayo,86,san_francisco,86755,vende_en_linea,transformacion,no_asociado,no_asociado,0,1,202186755
63358,2021,2021-02-12,putumayo,86,santiago,86760,vende_en_linea,transformacion,no_asociado,no_asociado,0,3,202186760
63359,2021,2021-02-12,guaviare,95,san_jose_del_guaviare,95001,vende_en_linea,transformacion,no_asociado,no_asociado,0,24,202195001


Now we will examine the last table, Components Dimensions:

In [52]:
comp_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Anio_corte                           4404 non-null   int64  
 1   Municipio                            4404 non-null   object 
 2   Código                               4404 non-null   int64  
 3   Departamento                         4404 non-null   object 
 4   Grupo_dotaciones                     4404 non-null   object 
 5   Categoría_de_ruralidad               4404 non-null   object 
 6   Cobertura_media_neta                 4404 non-null   float64
 7   SABER_11_Matematicas                 4404 non-null   float64
 8   SABER_11_Lenguaje                    4404 non-null   float64
 9   Cobertura_Transición                 4404 non-null   float64
 10  Cobertura_salud                      4404 non-null   float64
 11  Vacunación_Pentavalente       

In [53]:
comp_dim.columns = remove_tildes(comp_dim.columns.str.lower())
comp_dim.columns

Index(['anio_corte', 'municipio', 'codigo', 'departamento', 'grupo_dotaciones',
       'categoria_de_ruralidad', 'cobertura_media_neta',
       'saber_11_matematicas', 'saber_11_lenguaje', 'cobertura_transicion',
       'cobertura_salud', 'vacunacion_pentavalente', 'mortalidad_infantil',
       'cobertura_electrica_rural', 'cobertura_internet',
       'cobertura_acueducto', 'cobertura_alcantarillado', 'hurtos',
       'hurtos_x_10000_hab', 'homicidios', 'homicidios_x_10000_hab',
       'violencia_intrafamiliar', 'violencia_intrafamiliar_x_10000_hab'],
      dtype='object')

In [54]:
comp_dim["municipio"] = remove_tildes(comp_dim["municipio"].str.lower())
comp_dim["departamento"] = remove_tildes(comp_dim["departamento"].str.lower())
comp_dim["grupo_dotaciones"] = comp_dim["grupo_dotaciones"].str.lower()
comp_dim["categoria_de_ruralidad"] = remove_tildes(comp_dim["categoria_de_ruralidad"].str.lower())

Anything unusual here in the dtypes of the columns, except for the rural electric coverage variable, which is an `object` dtype. Let's examine this variable:

In [55]:
comp_dim["cobertura_electrica_rural"].unique()

array(['1', '0.98', '0.95', '0.97', '0.99', '0.88', '0.54', '0.91', '0.9',
       '0.94', '0.7', '0.85', '0.96', '0.89', '0.77', '0.78', '0.93',
       '0.68', '0.84', '0.79', '0.71', '0.76', '0.81', '0.67', '0.92',
       '0.8', '0.64', '0.82', '0.43', '0.55', '0.25', '0.52', '0.75',
       '0.83', '0.61', '0.35', '0.42', '0.74', '0.6', '0.65', '0.19',
       '0.86', '0.26', '0.63', '0.44', '0.46', '0.69', '0.48', '0.2',
       '0.09', '_-______', '0.87', '0.58', '0.5', '0.45', '0.4', '0.41',
       '0.13', '0.08', '0.56', '0.73', '0.72', '0.38', '0.24', '0.51',
       '0.62', '0.53', '0.15', '0.59', '0.29', '0.39', '0.37', '0.16',
       '0.66', '0.57', '0.1', '0.47', '0.02', '0.49', '0.11', '0.06',
       '0.3', '0.33', '0.28', '0.22', '0.36', '0.14', '0.27', '0.21',
       '0.17', '0.03', '0.31', '0', '0.07', '0.04', '0.34', '0.23', nan,
       '0.18', '0.971', '0.851', '0.984', '0.972', '0.983', '0.976',
       '0.743', '0.966', '0.941', '0.985', '0.853', '0.973', '0.982',
       

We already see there are an interesting element '_-______' which cannot be converted to numeric whatsoever. This element surely represent missing data, and has this particular form since in the basic cleaning process I removed all the blanks and replaced them with underscores `_`. Let's see which rows have this element in the column: 

In [56]:
nonfloats=[]
for i in range(len(comp_dim)):
    try:
        float(comp_dim["cobertura_electrica_rural"][i])
    except ValueError:
        nonfloats.append(i)
print(nonfloats)
print(comp_dim[["anio_corte","municipio","departamento","cobertura_electrica_rural"]].loc[nonfloats])

[267, 456, 599, 663, 1086]
      anio_corte            municipio departamento cobertura_electrica_rural
267         2016                pisba       boyaca                  _-______
456         2016               tuchin      cordoba                  _-______
599         2016  san_jose_del_palmar        choco                  _-______
663         2016            concordia    magdalena                  _-______
1086        2016          providencia   san_andres                  _-______


And let's get rid of them (by replacing them with na's):

In [57]:
cobertura_electrica_floats=[]
for i in range(len(comp_dim)):
    try:
        cobertura_electrica_floats.append(float(comp_dim["cobertura_electrica_rural"][i]))
    except ValueError:
        cobertura_electrica_floats.append(np.nan)
comp_dim["cobertura_electrica_rural"] = cobertura_electrica_floats

Let's check again and see whether the problem was fixed:

In [58]:
comp_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   anio_corte                           4404 non-null   int64  
 1   municipio                            4404 non-null   object 
 2   codigo                               4404 non-null   int64  
 3   departamento                         4404 non-null   object 
 4   grupo_dotaciones                     4404 non-null   object 
 5   categoria_de_ruralidad               4404 non-null   object 
 6   cobertura_media_neta                 4404 non-null   float64
 7   saber_11_matematicas                 4404 non-null   float64
 8   saber_11_lenguaje                    4404 non-null   float64
 9   cobertura_transicion                 4404 non-null   float64
 10  cobertura_salud                      4404 non-null   float64
 11  vacunacion_pentavalente       

Alright! So now let's describe those numeric variables:

In [59]:
(comp_dim["anio_corte"].unique())

array([2016, 2017, 2018, 2019])

In [60]:
comp_dim.describe()

,anio_corte,codigo,cobertura_media_neta,saber_11_matematicas,saber_11_lenguaje,cobertura_transicion,cobertura_salud,vacunacion_pentavalente,mortalidad_infantil,cobertura_electrica_rural,cobertura_internet,cobertura_acueducto,cobertura_alcantarillado,hurtos,hurtos_x_10000_hab,homicidios,homicidios_x_10000_hab,violencia_intrafamiliar,violencia_intrafamiliar_x_10000_hab
count,4404.000000,4404.000000,4404.000000,4404.000000,4404.000000,4404.000000,4404.000000,4404.000000,4404.000000,4398.000000,4404.000000,4404.000000,4404.000000,4404.000000,4404.000000,4404.000000,4404.000000,4404.000000,4404.000000
mean,2017.500000,37671.950954,30.172471,47.950800,49.880186,39.095985,40.670611,45.912805,18.863228,0.879465,1.775996,0.571314,0.388369,254.149637,17.872695,11.413261,2.480690,80.103315,10.004996
std,1.118161,25705.991227,22.069349,4.823078,3.592606,26.706587,41.569208,46.425937,9.315656,0.188705,3.925243,0.298542,0.281378,3363.223109,21.107681,57.512463,3.171105,934.408670,10.011659
min,2016.000000,5001.000000,0.024000,0.000000,0.000000,0.130000,0.170000,0.170000,1.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2016.750000,15646.000000,1.000000,44.750000,47.703000,4.075000,0.830000,0.950000,13.510000,0.860000,0.014000,0.340000,0.160000,4.000000,4.460000,0.000000,0.000000,3.000000,3.110000
50%,2017.500000,25781.000000,33.340000,48.195500,50.129000,43.920000,9.600000,8.350000,17.160000,0.960000,0.100000,0.570000,0.340000,12.000000,10.560000,2.000000,1.590000,9.000000,7.270000
75%,2018.250000,63302.000000,45.792500,51.356250,52.351750,58.322500,82.600000,92.200000,21.914000,0.990000,1.800000,0.832000,0.580000,44.000000,22.990000,7.250000,3.273055,23.000000,13.545086
max,2019.000000,99773.000000,100.000000,66.130000,64.130000,100.000000,100.000000,339.400000,87.910000,1.000000,68.600000,1.000000,1.000000,156435.000000,206.028787,1288.000000,31.630000,36179.000000,116.500000


Here the main theme to look at are those variables described as "coverages", since we already saw that some of these variables are in two forms: as percentage (0%-100%) as well as proportions (0-1), depending on the year. So we should convert these variables to the same form (either percentage or proportion). Here I choose to convert all of them to proportions since I think it will be easier ad more confortable to do math with them in this form. 

First we take all the variables described as "coverages":

In [61]:
variables = list(comp_dim.columns[comp_dim.columns.str.contains("cobertura", case=False)])+['vacunacion_pentavalente']

Then we try and convert every value above 1 to its corresponding value between 0 and 1 (we'll surely have to double-check this procedure with the original dataset):

In [62]:
for variable in variables:
    comp_dim[variable] = [comp_dim[variable][i]/100 if comp_dim[variable][i] > 1 else comp_dim[variable][i] for i in range(len(comp_dim))]

comp_dim[variables].describe()

,cobertura_media_neta,cobertura_transicion,cobertura_salud,cobertura_electrica_rural,cobertura_internet,cobertura_acueducto,cobertura_alcantarillado,vacunacion_pentavalente
count,4404.000000,4404.000000,4404.000000,4398.000000,4404.000000,4404.000000,4404.000000,4404.000000
mean,0.407100,0.535216,0.808769,0.879465,0.101718,0.571314,0.388369,0.910899
std,0.156045,0.164099,0.160228,0.188705,0.193781,0.298542,0.281378,0.137858
min,0.013100,0.051000,0.170000,0.000000,0.000000,0.000000,0.000000,0.157000
25%,0.307000,0.423000,0.710000,0.860000,0.012000,0.340000,0.160000,0.846000
50%,0.404900,0.532050,0.828000,0.960000,0.030000,0.570000,0.340000,0.938000
75%,0.497000,0.637000,0.945000,0.990000,0.097250,0.832000,0.580000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.394000


All seem good and sound. However, I did not see before that the `Vacunación_Pentavalente` column (which is described in the data dictionary as a percentage) has originally values above 100% (339% indeed!). We should examine which rows have values above 1 for that variable:

In [63]:
comp_dim[comp_dim["vacunacion_pentavalente"]>1]

,anio_corte,municipio,codigo,departamento,grupo_dotaciones,categoria_de_ruralidad,cobertura_media_neta,saber_11_matematicas,saber_11_lenguaje,cobertura_transicion,cobertura_salud,vacunacion_pentavalente,mortalidad_infantil,cobertura_electrica_rural,cobertura_internet,cobertura_acueducto,cobertura_alcantarillado,hurtos,hurtos_x_10000_hab,homicidios,homicidios_x_10000_hab,violencia_intrafamiliar,violencia_intrafamiliar_x_10000_hab
1107,2017,andes,5034,antioquia,g3,intermedios,0.3340,47.48,51.66,0.5431,0.820,1.012,14.45,0.98,0.061,0.51,0.41,73,15.66,30,6.43,75,16.09
1108,2017,angelopolis,5036,antioquia,g3,intermedios,0.3141,43.90,50.00,0.2966,0.473,1.017,13.51,0.98,0.022,0.79,0.77,11,11.94,4,4.34,17,18.45
1109,2017,angostura,5038,antioquia,g3,rural_disperso,0.4928,41.28,46.09,0.4074,0.906,1.192,27.41,0.97,0.027,0.54,0.10,14,12.57,9,8.08,27,24.24
1112,2017,anza,5044,antioquia,g4,rural_disperso,0.1895,45.61,49.00,0.3529,0.895,1.130,14.93,0.93,0.020,0.21,0.14,8,10.54,3,3.95,10,13.17
1119,2017,bello,5088,antioquia,g1,ciudades_y_aglomeraciones,0.3829,49.35,53.49,0.5337,0.849,1.019,8.77,1.00,0.207,1.00,1.00,1886,39.84,82,1.73,669,14.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2172,2017,trinidad,85430,casanare,g1,rural_disperso,0.2709,49.24,50.96,0.6080,0.733,1.004,29.83,0.55,0.023,0.55,0.55,30,19.39,1,0.65,13,8.40
2175,2017,colon,86219,putumayo,g3,intermedios,0.3990,52.62,54.25,0.4100,0.869,1.113,16.30,0.86,0.061,1.00,0.81,1,1.78,0,0.00,21,37.47
2184,2017,santiago,86760,putumayo,g5,rural,0.2454,47.98,50.57,0.2837,0.682,1.319,16.96,0.56,0.017,1.00,0.36,3,2.81,1,0.94,4,3.75
2186,2017,villagarzon,86885,putumayo,g1,rural,0.4337,48.23,50.98,0.5165,1.000,1.064,27.79,0.61,0.012,0.29,0.29,33,15.49,8,3.75,14,6.57


285 rows have indeed proportion values above 1, which should be impossible.

In [64]:
print(f'the year(s) that these anomalous values appear are: {comp_dim[comp_dim["vacunacion_pentavalente"]>1]["anio_corte"].unique()}')
print(f'The proportion of 2017 registers that these represent is {len(comp_dim[comp_dim["vacunacion_pentavalente"]>1])/len(comp_dim[comp_dim["anio_corte"]==2017])}')

the year(s) that these anomalous values appear are: [2017]
The proportion of 2017 registers that these represent is 0.25885558583106266


An interesting feature of those registers is that all of them are from the 2017 year. Also, they represent over 25% of the 2017 data, so we cannot dismiss them easily. Well, I think we will have to think also about this.

Finally, let's create the primary key for this table: 

In [65]:
comp_dim["id"] = comp_dim["anio_corte"].astype("string")+comp_dim["codigo"].astype("string")
comp_dim

,anio_corte,municipio,codigo,departamento,grupo_dotaciones,categoria_de_ruralidad,cobertura_media_neta,saber_11_matematicas,saber_11_lenguaje,cobertura_transicion,cobertura_salud,vacunacion_pentavalente,mortalidad_infantil,cobertura_electrica_rural,cobertura_internet,cobertura_acueducto,cobertura_alcantarillado,hurtos,hurtos_x_10000_hab,homicidios,homicidios_x_10000_hab,violencia_intrafamiliar,violencia_intrafamiliar_x_10000_hab,id
0,2016,medellin,5001,antioquia,c,ciudades_y_aglomeraciones,0.559,49.260,51.700,0.737,1.000,0.971,11.120,1.000,0.214,0.970,0.920,13871,55.780000,536,2.160000,2374,9.550000,20165001
1,2016,abejorral,5002,antioquia,g3,rural,0.329,48.100,51.280,0.590,0.882,0.813,13.160,0.980,0.046,0.370,0.280,28,14.590000,7,3.650000,13,6.770000,20165002
2,2016,abriaqui,5004,antioquia,g3,rural_disperso,0.522,46.940,50.710,0.848,0.996,0.852,16.950,0.950,0.030,0.290,0.290,1,4.820000,0,0.000000,0,0.000000,20165004
3,2016,alejandria,5021,antioquia,g2,rural,0.330,47.200,49.890,0.979,1.000,0.831,16.790,0.970,0.079,0.500,0.310,5,14.560000,1,2.910000,3,8.730000,20165021
4,2016,amaga,5030,antioquia,g2,intermedios,0.421,45.750,49.870,0.401,0.788,0.979,12.560,0.990,0.074,1.000,0.340,62,20.830000,9,3.020000,39,13.100000,20165030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4399,2019,taraira,97666,vaupes,g2,rural_disperso,0.083,45.333,47.095,0.286,0.562,0.600,85.616,0.126,0.009,0.977,0.977,2,8.628128,0,0.000000,11,47.454702,201997666
4400,2019,puerto_carrenio,99001,vichada,g1,rural_disperso,0.260,46.873,49.463,0.690,1.000,1.000,35.981,0.324,0.065,0.613,0.000,209,102.080688,15,7.326365,45,21.979095,201999001
4401,2019,la_primavera,99524,vichada,g4,rural_disperso,0.296,49.623,51.142,0.482,0.918,0.815,36.501,0.000,0.001,0.485,0.000,10,10.205123,0,0.000000,6,6.123074,201999524
4402,2019,santa_rosalia,99624,vichada,g2,rural_disperso,0.177,44.700,46.675,0.317,0.818,0.988,24.618,0.000,0.004,0.701,0.000,2,4.842615,1,2.421308,2,4.842615,201999624


At last, we would like to know of differences in spelling of either departments or municipalities. Since we already know that the MDM and the Comp_dim databases share the spelling, we will compare only the MDM to the investments databases:

In [66]:
munic_cods_nombres=mdm.merge(investments, left_on="codigo", right_on="municipio_cod")[["municipio", "codigo", "municipio_nombre", "municipio_cod"]].drop_duplicates()
munic_cods_nombres[~(munic_cods_nombres["municipio"]==munic_cods_nombres["municipio_nombre"])]

,municipio,codigo,municipio_nombre,municipio_cod
16920,peniol,5541,peðol,5541
33044,"bogota,_d.c.",11001,bogota_d.c.,11001
53656,guican,15332,gsican,15332
69208,togui,15816,togsi,15816
84448,guachene_,19300,guachene,19300
148888,chivolo,47170,chibolo,47170
164512,chachagui,52240,chachagsi,52240
169060,magui,52427,magsi,52427
199496,guepsa,68327,gsepsa,68327


In [67]:
depa_cods_nombres=mdm.merge(investments, left_on="codigo", right_on="municipio_cod")[["departamento", "departame_nombre", "departame_cod"]].drop_duplicates()
depa_cods_nombres[~(depa_cods_nombres["departamento"]==depa_cods_nombres["departame_nombre"])]

,departamento,departame_nombre,departame_cod
33044,bogota_d.c.,bogota,11
215720,tolima_,tolima,73
246604,san_andres,san_andres_y_providencia,88


Since there are few misspellings among the databases, we will proceed to write down new CSV files for each database and finish the cleaning (correcting the misspelings) in Bash:

In [68]:
mdm.to_csv("../data/mdm_clean.csv", index=False)
comp_dim.to_csv("../data/comp_dim_clean.csv", index=False)
investments.to_csv("../data/mintic_investments_clean.csv", index=False)

Let's check that there are no more misspellings in the departments/municipalities names:

In [70]:
mdm = pd.read_csv("../data/mdm_clean.csv")
comp_dim = pd.read_csv("../data/comp_dim_clean.csv", header=1)
investments = pd.read_csv("../data/mintic_investments_clean.csv")

In [71]:
depa_cods_nombres=mdm.merge(investments, left_on="codigo", right_on="municipio_cod")[["departamento", "departame_nombre", "departame_cod"]].drop_duplicates()
depa_cods_nombres[~(depa_cods_nombres["departamento"]==depa_cods_nombres["departame_nombre"])]

,departamento,departame_nombre,departame_cod


In [72]:
munic_cods_nombres=mdm.merge(investments, left_on="codigo", right_on="municipio_cod")[["municipio", "codigo", "municipio_nombre", "municipio_cod"]].drop_duplicates()
munic_cods_nombres[~(munic_cods_nombres["municipio"]==munic_cods_nombres["municipio_nombre"])]

,municipio,codigo,municipio_nombre,municipio_cod
